# Retrieve Dataset

This notebook is used to capture a dataset.
The dataset contains RGB and depth images of both cameras, zivid and realsense.
Besides the images, the dataset stores the intrisic camera matrizes and distortion coefficients of both RGB lenses.

In [ ]:
# import dependencies
import time
import numpy as np
from pathlib import Path
# import required modules
from models.camera_interfaces.real_sense import RealSense
from models.camera_interfaces.zivid import Zivid
from models.dataset.dataset_container import DatasetContainer

In [ ]:
resource_path = Path("../../resources")

class Args:
    picture_amount = 3                  # how many frames to collect
    delay = 500                         # delay between exposures in ms
    zivid_configuration_path: resource_path / Path("calibrations/zivid.yml") # zivid settings
    dataset_path = resource_path / Path("images/uncalibrated/dataset_0.npz") # where the captures dataset should be saved
args = Args()

# Connect to cameras

In [ ]:
# make sure both instances are destroyed. Otherwise they are still connected to the cameras
%reset_selective -f real_sense
%reset_selective -f zivid
# instantiate and connect to cameras
real_sense = RealSense()
RealSense.connect()

zivid = Zivid()
zivid.connect()
zivid.configure_manual(args.zivid_configuration_path)

print("Connected to both cameras")

rs_intrinsics = real_sense.get_camera_matrix_and_distortion()
zv_intrinsics = zivid.get_camera_matrix_and_distortion()

dataset_container = DatasetContainer()
dataset_container.load_from_intrinsics(*rs_intrinsics, *zv_intrinsics)

## Collect Pictures

In [ ]:
for i in range(args.picture_amount):
    rs_frames = real_sense.collect_frame()
    zv_frames = zivid.collect_frame()
    dataset_container.append(*rs_frames, *zv_frames)

    time.sleep(args.delay / 1000.0)

print("Pictures Retrieved")

dataset_container.save(args.dataset_path)
print(f"Dataset saved to {args.dataset_path}")